In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import gc
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

DATA_DIR = '/content/drive/MyDrive/datathon_2022/data'

## 1. CPS Data

In [ ]:
cps_data = pd.read_csv(os.path.join(DATA_DIR, 'cps_data/cps_data.csv'), index_col=0)

In [ ]:
cps_data.head()

In [ ]:
zip_map = pd.read_csv(os.path.join(DATA_DIR, 'cps_data/ZIP-COUNTY-FIPS.csv'))

zip_map['ZIP'] = zip_map['ZIP'].astype(str)
zip_map['ZIP_len'] = zip_map['ZIP'].str.len()
zip_map.loc[zip_map['ZIP_len']==4, 'ZIP'] = '0'+zip_map.loc[zip_map['ZIP_len']==4, 'ZIP']
zip_map.loc[zip_map['ZIP_len']==3, 'ZIP'] = '00'+zip_map.loc[zip_map['ZIP_len']==4, 'ZIP']
zip_map['zip3'] = zip_map['ZIP'].apply(lambda x: str(x)[:3])

zip_map = zip_map.drop(columns=['ZIP', 'ZIP_len', 'CLASSFP']).drop_duplicates()
num_zip3_per_state = zip_map.groupby('STATE', as_index=False)['zip3'].count()

In [ ]:
cps_data = cps_data.merge(zip_map, left_on='COUNTY', right_on='STCOUNTYFP', how='left')
# cps_data = cps_data.dropna(subset={'zip3'})

In [ ]:
state_map = pd.read_csv(os.path.join(DATA_DIR, 'cps_data/us-state-ansi-fips.csv'))
state_map[' stusps'] = state_map[' stusps'].apply(lambda x: x.strip())

cps_data = cps_data.merge(state_map, left_on='STATEFIP', right_on=' st', how='left')
cps_data['STATE'] = cps_data['STATE'].fillna(cps_data[' stusps'])
cps_data = cps_data.drop(columns = ['stname', ' st', ' stusps', 'STATEFIP'])

In [ ]:
cps_data.head()

### 1.1 Income

In [ ]:
income_decode = """
112                 $500 - 999
120                 $1,000 - 1,999
121                 $1,000 - 1,499
122                 $1,500 - 1,999
130                 $2,000 - 2,999
131                 $2,000 - 2,499
132                 $2,500 - 2,999
140                 $3,000 - 3,999
141                 $3,000 - 3,499
142                 $3,500 - 3,999
150                 $4,000 - 4,999
200                 $5,000 - 7,999
210                 $5,000 - 7,499
220                 $5,000 - 5,999
230                 $6,000 - 7,999
231                 $6,000 - 7,499
232                 $6,000 - 6,999
233                 $7,000 - 7,499
234                 $7,000 - 7,999
300                 $7,500 - 9,999
310                 $7,500 - 7,999
320                 $8,000 - 8,499
330                 $8,500 - 8,999
340                 $8,000 - 8,999
350                 $9,000 - 9,999
400                 $10,000 - 14,999
410                 $10,000 - 10,999
420                 $11,000 - 11,999
430                 $10,000 - 12,499
440                 $10,000 - 11,999
450                 $12,000 - 12,999
460                 $12,000 - 14,999
470                 $12,500 - 14,999
480                 $13,000 - 13,999
490                 $14,000 - 14,999
500                 $15,000 - 19,999
510                 $15,000 - 15,999
520                 $16,000 - 16,999
530                 $17,000 - 17,999
540                 $15,000 - 17,499
550                 $17,500 - 19,999
560                 $18,000 - 19,999
600                 $20,000 - 24,999
700                 $25,000 - 49,999
710                 $25,000 - 29,999
720                 $30,000 - 34,999
730                 $35,000 - 39,999
740                 $40,000 - 49,999
810                 $50,000 - 74,999
820                 $50,000 - 59,999
830                 $60,000 - 74,999
841                 $75,000 - 99,999
842                 $100,000 - 149,999
"""
income_map = list(filter(None,[list(filter(None,i.split(' '))) for i in income_decode.split('\n')]))
income_map = pd.DataFrame(income_map, columns=['FAMINC', 'lower', 'x', 'upper']).drop(columns=['x'])
income_map = income_map.replace({'\$': '', ',':''}, regex=True)
income_map['estimate'] = (income_map['lower'].astype(int) + income_map['upper'].astype(int))/2
income_map = income_map.set_index('FAMINC')['estimate'].to_dict()

# additional decode
income_map["100"] = 2500
income_map["843"] = 200000

In [ ]:
income_df = cps_data[(cps_data['PERNUM'] == 1) & (cps_data['FAMINC'] < 900)]
income_df['FAMINC'] = income_df['FAMINC'].astype(str).replace(income_map)
income_df = income_df.groupby(['YEAR', 'STATE'], as_index=False)['FAMINC'].mean()
# income_df = income_df.groupby(['YEAR', 'zip3'], as_index=False)['FAMINC'].mean()

### 1.2 Race

In [ ]:
cps_data['is_black'] = (cps_data['RACE'].isin([200,801,805,806,807])).astype(int)
cps_data['is_asian'] = (cps_data['RACE'].isin([650,651,652,809])).astype(int)
cps_data['is_white'] = (cps_data['RACE'].isin([100,802,803,804])).astype(int)
cps_data['is_amind'] = (cps_data['RACE'].isin([300])).astype(int)
race_df = cps_data.groupby(['YEAR', 'STATE'], as_index=False)[['is_black', 'is_asian', 'is_white','is_amind']].mean()
# race_df = cps_data.groupby(['YEAR', 'zip3'], as_index=False)[['is_black', 'is_asian', 'is_white', 'is_amind']].mean()

### 1.3 Education

In [ ]:
educ_df = cps_data[(cps_data['PERNUM'] == 1) & (cps_data['EDUC'] != 999)]

educ_df['is_college'] = (educ_df['EDUC'] >= 80).astype(int)
educ_df = educ_df.groupby(['YEAR', 'STATE'], as_index=False)[['is_college']].mean()
# educ_df = educ_df.groupby(['YEAR', 'zip3'], as_index=False)[['is_college']].mean()

### 1.4 Population

In [ ]:
pop_df.head()

In [ ]:
pop_df = cps_data.groupby(['YEAR', 'STATE'], as_index=False)['CPSIDP'].count()
# pop_df = cps_data.groupby(['YEAR', 'zip3'], as_index=False)['CPSIDP'].count()

In [ ]:
demographic_df = income_df.merge(race_df, how='outer').merge(educ_df, how='outer').merge(pop_df, how='outer')

In [ ]:
demographic_df['race_sum'] = demographic_df['is_black']+demographic_df['is_asian']+demographic_df['is_white']
demographic_df = demographic_df[demographic_df['race_sum'] > 0.1]

In [ ]:
demographic_df.head()

In [ ]:
demographic_df = demographic_df.merge(num_zip3_per_state, how='left')
demographic_df['CPSIDP_zip'] = demographic_df['CPSIDP'] / demographic_df['zip3']
demographic_df = demographic_df.drop(columns=['zip3'])

In [ ]:
demographic_df.to_csv(os.path.join(DATA_DIR, 'cps_data/demographic_cleaned_states.csv'), index=False)

## QC

In [ ]:
demographic_df = pd.read_csv(os.path.join(DATA_DIR, 'cps_data/demographic_cleaned_states.csv'))

In [ ]:
demographic_df.head()